### The JSON outline of the research outline follows this structure:

{ 
   "title": "1. Introduction",
   "subsections": [
       {
           "title": "1.1 The Global Food Challenge and the Role of Irrigation",
           "point1": {
               "text": "...",
               "query1": "...",
               "query2": "...",
               ...
           },
           "point2": {
               "text": "...", 
               "query1": "...",
               "query2": "...",
               ...
           },
           ...
       },
       ...
   ]
}

#### The mapping from JSON to Excel is as follows:

1. The `point_text` value itself, mapped to the `text` field.
2. One or more `query` entries, mapped to the `query` field.
3. For each `query` entry, there will be the following dependent fields:
   * `doi`
   * `title` 
   * `full_text`
   * `bibtex`
   * `pdf_location`
   * `journal`
   * `citation_count`
   * `relevance_score`

To reconstruct the Excel table from the JSON, for each `point_text`, its value should be repeated for each associated `query`. This means that if there are 7 queries for a particular `point_text`, that `point_text` value should be repeated 7 times in the Excel table, with each repetition accompanied by the corresponding `query` and its dependent fields.

In [11]:
import os
import json
import pandas as pd

# Set the path to the documents folder
documents_folder = "documents"

# Create an Excel writer object
#output_file = documents_folder + "/master.xlsx"
#writer = pd.ExcelWriter(output_file, engine="openpyxl")

# Iterate over the JSON files in the documents folder
for filename in os.listdir(documents_folder):
    if filename.startswith("outline_") and filename.endswith(".json"):
        file_path = os.path.join(documents_folder, filename)
        
        # Extract the sheet name from the filename
        sheet_name = filename.split("_")[1].split(".")[0]
        
        # Load the JSON data from the file
        with open(file_path, "r") as file:
            json_data = json.load(file)
        
        # Create an empty list to store the data for the current JSON file
        data_list = []
        
        # Extract the data from the JSON structure
        for section in json_data:
            document_title = section["title"]
            for subsection in section.get("subsections", []):
                section_title = subsection["title"]
                for point_key, point_data in subsection.items():
                    if point_key.startswith("point"):
                        for query_key, query_data in point_data.items():
                            if query_key.startswith("query"):
                                data_list.append({
                                    "document_title": document_title,
                                    "section_title": section_title,
                                    "point_text": point_data["text"],
                                    "query": query_data,
                                    "doi": "",
                                    "title": "",
                                    "full_text": "",
                                    "bibtex": "",
                                    "pdf_location": "",
                                    "journal": "",
                                    "citation_count": "",
                                    "relevance_score": ""
                                })
        
        # Create a DataFrame from the data list
        df = pd.DataFrame(data_list)
        
        # Reorder the columns
        column_order = [
            "document_title",
            "section_title",
            "point_text",
            "query",
            "doi",
            "title",
            "full_text",
            "bibtex",
            "pdf_location",
            "journal",
            "citation_count",
            "relevance_score"
        ]
        df = df[column_order]
        
        # Save the DataFrame to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

# Save the Excel file
writer.close()
print(f"Data from JSON files loaded and saved to {output_file}.")

Data from JSON files loaded and saved to documents/master.xlsx.


In [ ]:
import openpyxl
import asyncio
import json
from llm_api_handler import LLM_APIHandler
from prompts import return_best_results

class ExcelQueryProcessor:
    def __init__(self, file_path, api_key_path):
        self.workbook = openpyxl.load_workbook(file_path)
        self.api_handler = LLM_APIHandler(api_key_path)

    async def process_queries(self, sheet_name, input_columns, output_columns, new_columns=None, match_column=None, match_strings=None, skip_columns=None):
        sheet = self.workbook[sheet_name]
        query_data = []

        for row in sheet.iter_rows(values_only=True):
            if match_column and match_strings:
                match_column_index = sheet.column_names.index(match_column)
                if row[match_column_index] not in match_strings:
                    continue

            if skip_columns:
                skip = False
                for col in skip_columns:
                    col_index = sheet.column_names.index(col)
                    if not row[col_index]:
                        skip = True
                        break
                if skip:
                    continue

            query_values = {col: row[sheet.column_names.index(col)] for col in input_columns}
            query_data.append({"row": row[0], "query": query_values})

        results = await self.process_queries_async(query_data, output_columns, new_columns)

        if new_columns:
            for col in new_columns:
                if col not in sheet.column_names:
                    sheet.column_dimensions[openpyxl.utils.get_column_letter(sheet.max_column + 1)].width = 20
                    sheet.cell(row=1, column=sheet.max_column, value=col)

        for result in results:
            row = result["row"]
            response = result["response"]
            for paper in response:
                for col, value in paper.items():
                    if col in output_columns:
                        col_index = sheet.column_names.index(col)
                        sheet.cell(row=row, column=col_index + 1, value=value)
                    elif col in new_columns:
                        col_index = sheet.max_column
                        sheet.cell(row=row, column=col_index, value=value)

        self.workbook.save(file_path)

    async def process_queries_async(self, query_data, output_columns, new_columns):
        tasks = []
        for query in query_data:
            task = asyncio.ensure_future(self.process_single_query(query, output_columns, new_columns))
            tasks.append(task)

        results = await asyncio.gather(*tasks)
        return results

    async def process_single_query(self, query, output_columns, new_columns):
        while True:
            outline = query["query"].get("outline", "")
            review_intention = query["query"].get("review_intention", "")
            point = query["query"].get("point", "")
            research_papers = query["query"].get("research_papers", "")

            prompt = return_best_results(outline, review_intention, point, research_papers)
            response = await self.api_handler.generate_gemini_content(prompt)

            try:
                response_json = json.loads(response)
                return {"row": query["row"], "response": response_json}
            except json.JSONDecodeError:
                print(f"Invalid JSON response. Retrying query: {prompt}")
                
                
# Usage example
file_path = "master.xlsx"
api_key_path = "api_keys.json"
sheet_name = "Sheet1"
input_columns = ["outline", "review_intention", "point", "research_papers"]
output_columns = ["doi", "title", "citation_count", "relevance_score", "journal"]
new_columns = ["New_Column1", "New_Column2"]
match_column = "category"
match_strings = ["A", "B"]
skip_columns = ["skip_column1", "skip_column2"]

processor = ExcelQueryProcessor(file_path, api_key_path)
asyncio.run(processor.process_queries(sheet_name, input_columns, output_columns, new_columns, match_column, match_strings, skip_columns))